In [1]:
import cv2
import numpy as np
import face_recognition
import os
import time
import smtplib, ssl
import random
import imaplib
import email
from email.header import decode_header
import webbrowser



In [2]:
#send mail code

smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "facerecognitionnitk@gmail.com"
#password = input("Type your password and press enter: ")
password = "facerecognition@1234"
receiver_email = "mathurshipra33@gmail.com"


# Try to log in to server and send email
message = """\
Subject:{}

Do you want to admit {}?.
"""
def send(person):
    message1=message.format(random.randint(1,1000),person)
    
    # Create a secure SSL context
    context = ssl.create_default_context()
    try:
        server = smtplib.SMTP(smtp_server,port)
        server.ehlo() # Can be omitted
        server.starttls(context=context) # Secure the connection
        server.ehlo() # Can be omitted
        server.login(sender_email, password)

        server.sendmail(sender_email, receiver_email, message1)
        # TODO: Send email here
    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.quit() 



In [3]:
#receive email block

def receive():
    
    # account credentials
    username = "facerecognitionnitk@gmail.com"
    password = "facerecognition@1234"
    # create an IMAP4 class with SSL 
    imap = imaplib.IMAP4_SSL("imap.gmail.com")
    # authenticate
    imap.login(username, password)

    status, messages = imap.select("INBOX")
    # number of top emails to fetch
    N = 1
    # total number of emails
    messages = int(messages[0])

    for i in range(messages, messages-N, -1):
        # fetch the email message by ID
        res, msg = imap.fetch(str(i), "(RFC822)")
        for response in msg:
            if isinstance(response, tuple):
                # parse a bytes email into a message object
                msg = email.message_from_bytes(response[1])
                # decode the email subject
                subject = decode_header(msg["Subject"])[0][0]
                if isinstance(subject, bytes):
                    # if it's a bytes, decode to str
                    subject = subject.decode()
                # email sender
                from_ = msg.get("From")
                # if the email message is multipart
                if msg.is_multipart():
                    # iterate over email parts
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        try:
                            body1 = part.get_payload(decode=True).decode()
                        except:
                            pass
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            break
    imap.close()
    imap.logout()

    body1 = str(body1).split('.')
    #print(body1)
    if body1[1] == 'dir="auto">Yes</div>' or body1[0] == 'Yes\n' or body1[0] == 'Yes' or body1[0] == 'yes' or body1[0] == 'yes\n':
        return "Person Admitted"
    else:
        return ("Person Not Admitted. Access Denied")
    


In [7]:
# Now, we make a list of the images we have and write a function that automatically gets the images and generates encodings for the face

path='SMARTLOCK' # Specifying the path from where to fetch the images inside the computer memory
Images=[]
Names=[]     #creating a list for images and their names,we fetch the names from the image name itself and do not hard code them separately


In [8]:
myList=os.listdir(path)     # Assigning the path to fetch the images 
for ppl in myList:
    current=cv2.imread(f'{path}/{ppl}')    # Reading the image in the path
    Images.append(current)
    Names.append(os.path.splitext(ppl)[0]) # Assigning the names and appending them to the names list 
print(Names)

['.ipynb_checkpoints', 'DarpitaMathur', 'KapilMathur', 'KusumMathur', 'NainaMathur', 'ShipraMathur']


In [11]:
# After loading the images,we need to find their encodings one by one ,for this we make a function

def Encod(Images):      # Passing the list of Images as parameter for finding encodings function
    EncodedList=[]      # List that will store all the encodings 
    for img in Images:
        img= cv2.cvtColor(img,cv2.COLOR_BGR2RGB)      #Image obtained initially is in BGR format,first we convert it into RGB format
        encoding=face_recognition.face_encodings(img)[0] # Encoding is found for each image in the list one by one and appended into the EncodedList
        EncodedList.append(encoding)

    return EncodedList  #return the list storing the encodings 
EncodeKnownFaces=Encod(Images)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-tswt8jna\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [10]:
 # The image to be comapred and recognised comes from the web cam connected,hence initialising the web cam using cv2
imgCapture=cv2.VideoCapture(0)  

while True:
    success, img=imgCapture.read()  # Reading and storing the images captured through webcam
    CompressImg=cv2.resize(img,(0,0),None,0.25,0.25)   #compressing the image to one fourth size 
    CompressImg = cv2.cvtColor(CompressImg,cv2.COLOR_BGR2RGB) 

    faceloc=face_recognition.face_locations(CompressImg)    # From the whole image,locating the face first 
    encodings=face_recognition.face_encodings(CompressImg,faceloc)   # Finding the encoding of each face captured through web cam in the compressed image

    for encodeface,facelocations in zip(encodings,faceloc):
        match=face_recognition.compare_faces(EncodeKnownFaces,encodeface) # comparing the face obtained from web cam with each face stored in the known list of faces to find if the person appearing at the doorstep is a known person or not
        facedis=face_recognition.face_distance(EncodeKnownFaces,encodeface)# Finds the distance/difference in values of the encoding of the image captured and the images present in the known list

        Index=np.argmin(facedis)  #This index refers to the index of the image from the list of images that we would take as reference which corressponds to the image having minimum difference in encoding values

        if match[Index]:   #now we check if on matching the obtained image from the reference image,we get true or not
            name=Names[Index]  # Assigning the name to be printed in case the face is recognised.Names were stored in the Names list
            y1,x2,y2,x1=facelocations
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
        # Above code multiplies the co-ordinates of the locations obtained by 4 because earlier the compression has reduced the image size to 1/4 the size,hence regaining the original co-ordinates
            cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2) #this draws a rectangle around the face in the captured image using the faceloc variable that has stored the co-ordinates of the location of the face
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)# printing the name onto the image according to the required postion,font and colour
            send(name)
            status=" "
            while status==" ":
                status=receive()
            print(status)
    
    
        else:
            text="Intruder/Unknown"
            y1,x2,y2,x1=facelocations
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,0),2)
            cv2.putText(img,text,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2) #cv2.putText() method is used to draw a text string on any image.
            send("imposter")
            status=" "
            while status==" ":
                status=receive()
            print(status)
    

    cv2.imshow('Image captured',img) #Displaying the image captured via webcam to the user
    cv2.waitKey(1)



error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-tswt8jna\opencv\modules\highgui\src\window.cpp:651: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
